<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/01_Energy_Convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pyscf py3DMol plotly==5.8.0 kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyscf import gto, scf, dft, mp, cc
import py3Dmol
import plotly.express as px

# Setting up our system

- We initialize the molecular (or solid) PySCF object with coordinates, basis, spin, and charge information
- We can check that things look right with [py3DMol](https://3dmol.csb.pitt.edu/). Use your mouse to move the molecular around!



In [3]:
mol_xyz = """O        0.00000    0.11779   0.00000
             H        0.75545   -0.47116   0.00000
             H       -0.75545   -0.47116   0.00000"""
mol = gto.M(atom=mol_xyz, basis="def2svp", verbose=4)

System: uname_result(system='Linux', node='0839eeb389d5', release='5.4.188+', version='#1 SMP Sun Apr 24 10:03:06 PDT 2022', machine='x86_64', processor='x86_64')  Threads 2
Python 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
numpy 1.21.6  scipy 1.4.1
Date: Tue May 31 15:12:06 2022
PySCF version 2.0.1
PySCF path  /usr/local/lib/python3.7/dist-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 O      0.000000000000   0.117790000000   0.000000000000 AA    0.000000000000   0.222590840213   0.000000000000 Bohr
[INPUT]  2 H      0.755450000000  -0.471160000000   0.000000000000 AA    1.427593600803  -0.890363360850   0.000000000000 Bohr
[INPUT]  3 H     -0.755450000000  -0.471160000000   0.000000000000 AA   -1.427593600803  -0.890363360850   0.000000000000 Bohr

nuclear repulsion = 9.1892

In [4]:
xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## [Hartree-Fock](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method)


* The starting point of the most of quantum chemistry
* We variationally optimize the orbitals for a single [Slater determinint](https://en.wikipedia.org/wiki/Slater_determinant)
* Working in the basis of atom-centered basis function we solve the [Roothaan-Hall](https://en.wikipedia.org/wiki/Roothaan_equations) equations:

$\textbf{FC} = \textbf{SC} \epsilon$

* $\textbf{F}$ is the [Fock matrix]()
* $\textbf{C}$ is the molecular orbital coefficient matrix
* $\textbf{S}$ is the atomic orbital overlap matrix
* $\epsilon$ is the vector of molecular orbital energies





In [5]:
mymf = scf.RHF(mol).run()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /content/tmpujmsr75t
max_memory 4000 MB (current use 211 MB)
Set gradient conv threshold to 3.16228e-05
init E= -75.7797774091097
  HOMO = -0.478708376647229  LUMO = 0.0990356524184008
cycle= 1 E= -75.9225305322194  delta_E= -0.143  |g|= 0.445  |ddm|= 1.33
  HOMO = -0.41778520928291  LUMO = 0.175082517867731
cycle= 2 E= -75.9520234518777  delta_E= -0.0295  |g|= 0.225  |ddm|= 0.404
  HOMO = -0.50600267430301  LUMO = 0.171257824315094
cycle= 3 E= -75.9607437950686  delta_E= -0.00872  |g|= 0.0293  |ddm|= 0.149
  HOMO = -0.496600495366499  LUMO = 0.175945173483801
cycle= 4 E= -75.9609532674162  delta_E= -0.000209  |g|= 0.00613  |ddm|= 0.0197
  HOMO = -0.49792207

## Density Functional Theory

TODO

In [6]:
myrks = dft.RKS(mol, xc="wB97x").run()



******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /content/tmp87oxn2xt
max_memory 4000 MB (current use 219 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = wB97x
    J.-D. Chai and M. Head-Gordon, J. Chem. Phys. 128, 084106 (2008)
radial grids: 
    Treutler-Ahlrichs [JCP 102, 346 (1995); DOI:10.1063/1.469408] (M4) radial grids
    
becke partition: Becke, JCP 88, 2547 (1988); DOI:10.1063/1.454033
pruning grids: <function nwchem_prune at 0x7f96223973b0>
grids dens level: 3
symmetrized grids: False
atomic radii adjust function: <function treutler_atomic_radii_adjust at 0x7f9622397830>
small_rho_cutoff = 

## Møller–Plesset perturbation theory

TODO

In [7]:
mymp2 = mp.MP2(mymf).run()


******** <class 'pyscf.mp.mp2.MP2'> ********
nocc = 5, nmo = 24
max_memory 4000 MB (current use 263 MB)
E(MP2) = -76.1646134597338  E_corr = -0.20364346956817


## Coupled Cluster

TODO

In [8]:
mycc = cc.CCSD(mymf).run()


******** <class 'pyscf.cc.ccsd.CCSD'> ********
CC2 = 0
CCSD nocc = 5, nmo = 24
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 263 MB)
Init t2, MP2 energy = -76.1646134738225  E_corr(MP2) -0.203643483656848
Init E_corr(CCSD) = -0.203643483658391
cycle = 1  E_corr(CCSD) = -0.208598554996107  dE = -0.00495507134  norm(t1,t2) = 0.021702
cycle = 2  E_corr(CCSD) = -0.211811484967462  dE = -0.00321292997  norm(t1,t2) = 0.007521
cycle = 3  E_corr(CCSD) = -0.21285931356674  dE = -0.0010478286  norm(t1,t2) = 0.00285612
cycle = 4  E_corr(CCSD) = -0.2130091882508  dE = -0.000149874684  norm(t1,t2) = 0.000557348
cycle = 5  E_corr(CCSD) = -0.212993867867437  dE = 1.53203834e-05  norm(t1,t2) = 0.00018018
cycle = 6  E_corr(CCSD) = -0.212989712090878  dE = 4.15577656e-06  norm(t1,t2) = 5.09528e-05
cycle = 7  E_corr(CCSD) = -0.212991217881787  dE = -1.50579091e-06  norm(t1,t2) = 1.67188

In [9]:
e_ccsd_t = mycc.ccsd_t()

CCSD(T) correction = -0.003009509969022


# Analysis

* TODO

In [22]:
# Collect data
methods = ["HF", "DFT", "MP2", "CCSD", "CCSD(T)"]
energies = [mymf.e_tot, myrks.e_tot, mymp2.e_tot, mycc.e_tot, mycc.e_tot + e_ccsd_t]

# Plotting
fig = px.line(x=methods, y=energies, title="Jacob's Ladder", markers=True)
fig.update_layout(xaxis_title="Method", yaxis_title="Energy (Ha)")
fig.update_traces(marker_size=12)
fig.show() # It's interactive!